<a href="https://colab.research.google.com/github/kumarmohit0911/Breast_cancer_detector_model/blob/main/Breast_cancer_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json, os

kaggle_creds = {
    "username": "kumarmohit0911",
    "key": "KGAT_70d781b0a7eed54b863a6bcb5ab50e9c"
}

os.makedirs("/root/.kaggle", exist_ok=True)

with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_creds, f)

os.chmod("/root/.kaggle/kaggle.json", 0o600)

print("kaggle.json created successfully")


kaggle.json created successfully


In [4]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory


In [5]:
!kaggle datasets download -d paultimothymooney/breast-histopathology-images


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images
License(s): CC0-1.0
100% 3.09G/3.10G [00:47<00:00, 134MB/s]
100% 3.10G/3.10G [00:47<00:00, 69.3MB/s]


In [6]:
!unzip -q breast-histopathology-images.zip

In [7]:
import os
DATA_DIR = "IDC_regular_ps50_idx5"

image_path = []
labels = []

for patient_id in os.listdir(DATA_DIR):
  patient_path = os.path.join(DATA_DIR,patient_id)
  if not os.path.isdir(patient_path):
    continue

  for label in ["0","1"]:
    label_path = os.path.join(patient_path,label)
    if not os.path.isdir(label_path):
      continue

    for image_name in os.listdir(label_path):
      image_path.append(os.path.join(label_path,image_name))
      labels.append(int(label))


In [8]:
import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image

my_transform = transforms.Compose([
    transforms.ToPILImage,
    transforms.Resize((28,28)),
    transforms.RandomRotation(degrees=45),
    transforms.ColorJitter(brightness=0.5,contrast=0.5),
    transforms.RandomGrayscale(p=0.5),
    transforms.ToTensor()

])

In [9]:
from torch.utils.data import Dataset
from PIL import Image
class CustomDataset(Dataset):
  def __init__(self,image_path,labels,transform=my_transform):
    #convert to pytorch tensors
    self.image_path = image_path
    self.labels = labels
    self.transforms = transforms
  def __len__(self):
    return len(self.image_path)

  def __getitem__(self,index):
        # 1. Load image from disk
        image = Image.open(self.image_path[index]).convert("RGB")

        # 2. Get label
        label = self.labels[index]
        image,labels = train_dataset[10]
        plt.title(f"Label: {label}")
        plt.imshow(image)
        image=self.transform(image)

        return image, label

In [10]:
dataset = CustomDataset(image_path,labels)

In [20]:
from torch.utils.data import random_split
import random
random.seed(42)
random.shuffle(image_path)

dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

train_dataset,test_dataset = random_split(dataset,[train_size,test_size])

In [19]:
image,labels = train_dataset[1]
plt.title(f"Label: {label}")
plt.imshow(image)

RecursionError: maximum recursion depth exceeded

In [14]:
image,labels = train_dataset[10]
plt.title(f"Label: {label}")
plt.imshow(image)

RecursionError: maximum recursion depth exceeded

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset,batch_size = 32,shuffle = True,pin_memory = True)
test_loader = DataLoader(test_dataset,batch_size = 32,shuffle = False,pin_memory = True)